In [1]:
import sklearn.feature_extraction
import pandas as pd
# Read in a set of SQL statements from various sources
import os
basedir = 'data'
filelist = os.listdir(basedir) 
df_list = []
for file in filelist:
    df = pd.read_csv(os.path.join(basedir,file), sep='|||', names=['raw_sql'], header=None)
    df['type'] = 'legit' if file.split('.')[0] == 'legit' else 'malicious'
    df_list.append(df)
dataframe = pd.concat(df_list, ignore_index=True)
dataframe.dropna(inplace=True)
print dataframe['type'].value_counts()
dataframe.head()

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


malicious    12892
legit         1003
Name: type, dtype: int64


,raw_sql,type
0,'; exec master..xp_cmdshell 'ping 10.10.1.2'--,malicious
1,create user name identified by 'pass123',malicious
2,create user name identified by pass123 tempora...,malicious
3,"exec sp_addlogin 'name' , 'password'",malicious
4,"exec sp_addsrvrolemember 'name' , 'sysadmin'",malicious


In [2]:
dataframe.describe()

,raw_sql,type
count,13895,13895
unique,6436,2
top,ELSE 1/(SELECT 0 FROM DUAL) END) FROM DUAL),malicious
freq,12,12892


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split

X = dataframe.raw_sql
y = dataframe["type"]
vec_opts = {
    "ngram_range": (1, 1),  # allow n-grams of 1-4 words in length (32-bits)
    "analyzer": "word",     # analyze hex words
    "token_pattern": ".",  # treat two characters as a word (e.g. 4b)
    "min_df" : 0.1
}
v = TfidfVectorizer(**vec_opts)

In [12]:
import numpy as np

X = v.fit_transform(X)
features = v.get_feature_names()
features

[u' ',
 u'"',
 u'#',
 u"'",
 u'(',
 u')',
 u',',
 u'-',
 u'0',
 u'1',
 u'2',
 u'3',
 u'4',
 u'5',
 u'6',
 u'7',
 u'8',
 u'9',
 u'=',
 u'a',
 u'b',
 u'c',
 u'd',
 u'e',
 u'f',
 u'g',
 u'h',
 u'i',
 u'j',
 u'k',
 u'l',
 u'm',
 u'n',
 u'o',
 u'p',
 u'q',
 u'r',
 u's',
 u't',
 u'u',
 u'v',
 u'w',
 u'x',
 u'y']

In [13]:
from sklearn.feature_extraction.text import TfidfTransformer

idf_opts = {"use_idf": True}
idf = TfidfTransformer(**idf_opts)

In [14]:
y.size

13895

In [15]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.grid_search import GridSearchCV

clf = MultinomialNB().fit(X, y)
all_predictions = clf.predict(X)

//anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [16]:
from sklearn.metrics import classification_report
print classification_report(y, all_predictions)

             precision    recall  f1-score   support

      legit       0.98      0.62      0.76      1003
  malicious       0.97      1.00      0.98     12892

avg / total       0.97      0.97      0.97     13895

